Input
----

https://thefiddler.substack.com/p/can-you-permeate-the-pyramid

Output
----

**Solution to the fiddler**

A few observations:
- Since we cannot move upwards, each path from the top to the bottom must be a sequence of downward moves interleaved with sideways moves. 
- At each level of vertices, if you specify the isideput edge (comisideg from above) asided the output edge (going down), you will also uniquely specify the sideways path that should exist between these two downward moves. Because of the 1 dimensional nature of each level, and the clause that no vertex can be repeated, there cannot be 2 different ways of getting from one point to the other.
- So, it suffices to just specify which edges are used for the downward moves. This completely specifies the path from the top to the bottom.
- The number of edges between the seven levels are 2,4,6,6,4,2.
- So, the total number of paths is their product = ( 2 * 4 * 6 ) ^ 2 = 48 ^ 2 = 2304.




**Solution to the Extra Credit**

The extra credit is trickier because multiple paths are possible at each level, and the number of paths depends on the choice of input edge / start vertex and output edge / end vertex. So, we have to really count all the options.

Let's write some code for that, starting with generating the graph in question. We'll number vertices as (x, y,z) tuples. So, all the vertices of the same level have the same z value.

In [1]:
def make_bipyramid_graph(n):
    """
    Given a number n, return a graph with 2n+1 levels
    where each vertex is connected to its neighbors.
    """
    # Create an empty graph
    graph = {}
    
    # Iterate through each vertex
    for z in range(-n,n+1):
        for x in range(0,n-abs(z)+1):
            for y in range(0,n-abs(z)-x+1):
                # Our vertex is represented by a tuple (x,y,z)
                # where x y z  are the coordinates in the triangular grid
                t = (x,y,z)
                
                # Initialize the list of neighbors for vertex t
                graph[t] = []

                # Iterate through the possible neighbors in the triangular grid
                for dz in [-1,0,1]:
                    for dx in [-1,0,1]:
                        for dy in [-1,0,1]:
                            if (dx,dy,dz) != (0,0,0):
                                # Calculate the new coordinates
                                x2 = x + dx
                                y2 = y + dy
                                z2 = z + dz
                                t2 = (x2,y2,z2)
                                                            
                                # Check if the new coordinates are valid
                                coords_in_range = (0 <= x2 and 0 <= y2 and
                                                   -n <= z2 and z2 <= n and 
                                                    x2 + y2 <= n - abs(z2))

                                direct_neighbor = (abs(dx) + abs(dy) + abs(dz)) == 1

                                same_level_diagonal_neighbor = (dz == 0 and (dx + dy) == 0)

                                positive_diagonal_neighbor = (z >= 0 and z2 >= 0 and dz != 0 and (dx + dy) == -dz)
                                negative_diagonal_neighbor = (z <= 0 and z2 <= 0 and dz != 0 and (dx + dy) == dz)
                                    
                                #print(t, t2)
                                    
                                # Check if the new coordinates are in range and if they are a valid neighbor
                                # A neighbor is valid if it is a direct neighbor or a diagonal neighbor
                                if (coords_in_range and (direct_neighbor or
                                    same_level_diagonal_neighbor or positive_diagonal_neighbor or
                                    negative_diagonal_neighbor)):
                                        # Add the neighbor to the list
                                        graph[t].append(t2)
        
    return graph

G1 = make_bipyramid_graph(1)
G2 = make_bipyramid_graph(2)
G3 = make_bipyramid_graph(3)
print(len(G1), len(G2), len(G3))

G = {}
for i in range(10):
    G[i] = make_bipyramid_graph(i)

5 14 30


Graphs seem correct, as far as I can tell. Number of vertices is correct, and the edges also seemed to be correct for G1. 

Let's proceed with using these.

In [2]:

def count_paths(graph, start, end, path=[], allow_X_gt_0=True, allow_Y_gt_0=True, allow_Z_up=False, allow_Z_down=True):
    """
    Count the number of paths from start to end in the graph.
    """
    #print("start", start, "end", end, "path", path)

    # If we have reached the end vertex, return 1
    if start == end:
        return 1
    
    # If we have visited the vertex before, return 0
    if start in path:
        return 0
    
    # Add the current vertex to the path
    path = path + [start]
    
    # Initialize the number of paths to 0
    num_paths = 0
    
    # Iterate through each neighbor of the current vertex
    for neighbor in graph[start]:
        
        # Check if the neighbor is valid based on the constraints
        x_ok = (allow_X_gt_0 or neighbor[0] == 0)
        y_ok = (allow_Y_gt_0 or neighbor[1] == 0)
        z_up_ok = (allow_Z_up or neighbor[2] <= start[2])
        z_down_ok = (allow_Z_down or neighbor[2] >= start[2])

        #print("neighbor", neighbor, "x_ok", x_ok, "y_ok", y_ok, "z_ok", z_ok)

        # If the neighbor is valid, recursively count the paths from the neighbor to the end vertex
        if x_ok and y_ok and z_up_ok and z_down_ok:
            num_paths += count_paths(graph, neighbor, end, path, allow_X_gt_0, allow_Y_gt_0, allow_Z_up, allow_Z_down)
    
    return num_paths

In [3]:
count_paths(G1, (0,0,1), (0,0,-1))


15

This count is correct. So, looking good.

---

In [4]:
count_paths(G3, (0,0,3), (0,0,-3), allow_Y_gt_0=False)

2304

This is also correct. So continuing to looking good. (This part is simulating the 2D fiddler case in the 3D code by disallowing the Y dimension).

---

In [5]:
count_paths(G2, (0,0,2), (0,0,-2))

11475

I don't know if this is correct or not, but it seems plausible, since it is greater than 3 x 9 x 9 x 3 = 729.

---

In [6]:
#count_paths(G3, (0,0,3), (0,0,-3))
# 1093007025
# But takes 33 minutes to run

1093007025

Wow. That took 33 minutes to run, and I was about to cancel the run a few times, but it did finally complete.

I am going to take that answer as correct. :)

---

Note: While I see the sequence from the 2D case (1,4,64,2304) on OEIS ( https://oeis.org/A002454 ), I don't see the sequence from the 3D case (1,15,11475,1093007025)

---

Actually, I am going to try writing a bit more code to verify the answer.

In [7]:
from fractions import Fraction
def average_paths_per_level(graph, z):
    num_pairs = 0
    num_paths = 0
    for src in graph:
        for dst in graph:
            if src[2] == z and dst[2] == z:
                num_pairs += 1
                num_paths += count_paths(graph, src, dst, allow_Z_up=False, allow_Z_down=False)
    #print("num_pairs", num_pairs)
    #print("num_paths", num_paths)
    return Fraction(num_paths, num_pairs)

# average sideways paths_per_level in G3
S2 = average_paths_per_level(G3, 2)
S1 = average_paths_per_level(G3, 1)
S0 = average_paths_per_level(G3, 0)
print("S2", S2)
print("S1", S1)
print("S0", S0)

Total_Paths = (3 * S2 * 9 * S1 * 18 * S0 * 18 * S1 * 9 * S2 * 3) * 1.0

print(f"Total_Paths= {Total_Paths}")

S2 5/3
S1 37/6
S0 2603/50
Total_Paths= 1298898301.5


Well, the answer is the same order of magnitude, but not exactly the same.
The fact that this new answer is not an integer probably means that my approximation is not exact.
I guess the choices at each level are not independent, so just multiplying the numbers from each level may not be correct.
So, the other answer is more likely to be correct than this one.
Maybe I'll write something more ???

In [8]:
from functools import cache
@cache
def sideways_path_core(n, src, dst):
    """
    Count the number of (purely) sideways paths from src to dst in the graph.
    """
    graph = G[n]
    num_side_paths = count_paths(graph, src, dst, allow_Z_up=False, allow_Z_down=False)
    #print(f"sideways_paths_core({n}, {src}, {dst}) = {num_side_paths}")
    return num_side_paths

@cache
def sideways_paths(n, src, dst):
    """
    Count the number of (purely) sideways paths from src to dst in the graph.
    """
    # print(f"sideways_paths({n}, {src}, {dst}) called")
    if (src == dst):
        num_side_paths = 1
    else:
        # Reflect points to use the 3-way / 6-way symmetry at each level.
        # Reflect about x = n/2, y = n/2, and x = y.
        side = n - abs(src[2])     
        if (src[0] > (side - src[1])/2):
            src = (side - src[1] - src[0], src[1], src[2])
            dst = (side - dst[1] - dst[0], dst[1], dst[2])
            # print(f"A reflected src {src} dst {dst}")
        if (src[1] > (side - src[0])/2):
            src = (src[0], side - src[0] - src[1], src[2])
            dst = (dst[0], side - dst[0] - dst[1], dst[2])
            # print(f"B reflected src {src} dst {dst}")
        if (src[1] > src[0]):
            src = (src[1], src[0], src[2])
            dst = (dst[1], dst[0], dst[2])
            # print(f"C reflected src {src} dst {dst}")
        # And further ensure src <= dst for since the count is the same in both directions.
        if (src > dst):
            src, dst = dst, src
        num_side_paths = sideways_path_core(n, src, dst)
    # print(f"sideways_paths({n}, {src}, {dst}) = {num_side_paths}")
    assert(num_side_paths > 0)
    return num_side_paths

@cache
def downward_paths(n, src, dst):
    """
    Count the number of downward (starting) paths from src to dst in the graph.
    """
    #print(f"downward_paths({n}, {src}, {dst}) called")
    if (src == dst):
        return 1
    
    graph = G[n]
    num_down_paths = 0
    for neighbor in graph[src]:
        if neighbor[2] < src[2]:
            num_down_paths += all_paths(n, neighbor, dst)
    #print(f"downward_paths({n}, {src}, {dst}) = {num_down_paths}")
    return num_down_paths

@cache
def all_paths(n, src, dst):
    """
    Count the number of sideways and downward paths from src to dst in the graph.
    """
    if (src == dst):
        return 1
    
    graph = G[n]
    num_paths = 0
    for sidestep in graph:
        if sidestep[2] == src[2]:
            paths_to_sidestep =  sideways_paths(n, src, sidestep) 
            paths_down_from_sidestep = downward_paths(n, sidestep, dst)
            num_paths += paths_to_sidestep * paths_down_from_sidestep      
            
    #print(f"all_paths({n}, {src}, {dst}) = {num_paths}")
    return num_paths

d1 = all_paths(1, (0,0,1), (0,0,-1))
print(d1)
assert(d1 == 15)

d2 = all_paths(2, (0,0,2), (0,0,-2))
print(d2)
assert(d2 == 11475)

d3 = all_paths(3, (0,0,3), (0,0,-3))
print(d3)
assert(d3 == 1093007025)


15
11475
1093007025


Yay!!!!

This matches the original results exactly, runs in 0 seconds, and uses caching to speed things up.

While this reuses some of the original code, it is sufficiently different too. So, does give some more confidence in the reuslts.

-----

After seeing the solutions, and Tom Keith's numbers for higher values of N, I am curious to see if my code can calculate those values too.

It can calculate n=4 also easily, but is slow for n=5, so I am going to try to optimize my original code a bit more.

After a lot more optimization, I can calculate up to n=6, but not trying higher values.

In [9]:
import time
P = {}
for i in range(7):
    start_time = time.time()
    P[i] = all_paths(i, (0,0,i), (0,0,-i))
    end_time = time.time()
    delta_time = end_time - start_time
    print(f"downward_paths for n={i} = {P[i]} (took {delta_time:.2f} seconds)")


downward_paths for n=0 = 1 (took 0.00 seconds)
downward_paths for n=1 = 15 (took 0.00 seconds)
downward_paths for n=2 = 11475 (took 0.00 seconds)
downward_paths for n=3 = 1093007025 (took 0.00 seconds)
downward_paths for n=4 = 52244816853213675 (took 0.26 seconds)
downward_paths for n=5 = 6472823166678668309527843125 (took 38.11 seconds)
downward_paths for n=6 = 11561557982049161046080105648122197757331625 (took 23080.65 seconds)
